### Notebook này thực hiện phân đoạn (segmentation) với U-Net 3 lớp, trong đó ba lớp bao gồm tiền cảnh (foreground), hậu cảnh (background), và biên (border).

In [ ]:
# We import all our dependencies.
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../../')
from voidseg.models import Seg, SegConfig
import numpy as np
from csbdeep.utils import plot_history
from voidseg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from voidseg.utils.seg_utils import *
from voidseg.utils.compute_precision_threshold import compute_threshold, precision
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy import ndimage
import tensorflow as tf
import keras.backend as K
import urllib
import os
import zipfile

### Tải dữ liệu DSB2018.
Từ cuộc thi Kaggle 2018 Data Science Bowl, chúng tôi sử dụng cùng một tập hợp con dữ liệu được sử dụng tại đây, bao gồm một bộ sưu tập đa dạng các nhân tế bào được chụp bằng các kính hiển vi huỳnh quang khác nhau. Chúng tôi đã trích xuất 4870 ảnh mẫu có kích thước 128×128 từ tập huấn luyện và thêm nhiễu Gaussian với giá trị trung bình là 0 và độ lệch chuẩn (sigma) lần lượt là 10 (n10), 20 (n20), và 40 (n40). Notebook này hiển thị kết quả đối với các ảnh n40.

In [ ]:
# create a folder for our data
if not os.path.isdir('./data'):
    os.mkdir('data')

# check if data has been downloaded already
zipPath="data/DSB.zip"
if not os.path.exists(zipPath):
    #download and unzip data
    data = urllib.request.urlretrieve('https://owncloud.mpi-cbg.de/index.php/s/LIN4L4R9b2gebDX/download', zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("data")

Dữ liệu được tải xuống ở định dạng npz, và ô dưới đây sẽ trích xuất dữ liệu huấn luyện, xác thực và kiểm tra dưới dạng mảng numpy.

In [ ]:
trainval_data =  np.load('data/DSB/train_data/dsb2018_TrainVal40.npz')
test_data =  np.load('data/DSB/test_data/dsb2018_Test40.npz', allow_pickle=True)
train_images = trainval_data['X_train']
val_images = trainval_data['X_val']
test_images = test_data['X_test']

train_masks = trainval_data['Y_train']
val_masks = trainval_data['Y_val']
test_masks = test_data['Y_test']

In [ ]:
print("Shape of train_images: ", train_images.shape, ", Shape of train_masks: ", train_masks.shape)
print("Shape of val_images: ", val_images.shape, ", Shape of val_masks: ", val_masks.shape)
print("Shape of test_images: ", test_images.shape, ", Shape of test_masks: ", test_masks.shape)

### Chuẩn bị dữ liệu cho bước phân đoạn
Tiếp theo, chúng ta chuẩn hóa toàn bộ dữ liệu thô bằng giá trị trung bình và độ lệch chuẩn (std) của train_images thô. Sau đó, chúng ta xáo trộn các ảnh huấn luyện thô và Ground Truth (GT) tương ứng. Cuối cùng, chúng ta phân chia các cặp ảnh thô và GT tương ứng để mô phỏng trường hợp không có đủ dữ liệu huấn luyện được gán nhãn. Để thực hiện việc phân chia này, hãy chỉ định tham số fraction dưới đây. Giá trị này phải nằm trong khoảng từ 0 (không bao gồm) đến 100 (bao gồm).

In [ ]:
fraction = 2 # Fraction of annotated GT and raw image pairs to use during training. 
random_seed = 1 # Seed to shuffle training data (annotated GT and raw image pairs).


assert 0 <fraction<= 100, "Fraction should be between 0 and 100"
mean, std = np.mean(train_images), np.std(train_images)

X_normalized = normalize(train_images, mean, std)
X_val_normalized = normalize(val_images, mean, std)
X_test_normalized = normalize(test_images, mean, std)

X_shuffled, Y_shuffled = shuffle_train_data(X_normalized, train_masks, random_seed = random_seed)
X_frac, Y_frac = fractionate_train_data(X_shuffled, Y_shuffled, fraction = fraction)
print("Training Data \n..................")
X, Y_train_masks = augment_data(X_frac, Y_frac)
print("\n")
print("Validation Data \n..................")
X_val, Y_val_masks = augment_data(X_val_normalized, val_masks)

In [ ]:
X = X[...,np.newaxis]
Y = convert_to_oneHot(Y_train_masks)
X_val = X_val[...,np.newaxis]
Y_val = convert_to_oneHot(Y_val_masks)
print("Shape of train images: ", X.shape, ", Shape of train masks: ", Y.shape)
print("Shape of validation images: ", X_val.shape, ", Shape of validation masks: ", Y_val.shape)

In [ ]:
sl=0
plt.figure(figsize=(20,5))
plt.subplot(1,4,1)
plt.imshow(X_val[sl,...,0])
plt.title('Raw validation image')
plt.subplot(1,4,2)
plt.imshow(Y_val[sl,...,0])
plt.title('1-hot encoded background')
plt.subplot(1,4,3)
plt.imshow(Y_val[sl,...,1])
plt.title('1-hot encoded foreground')
plt.subplot(1,4,4)
plt.imshow(Y_val[sl,...,2])
plt.title('1-hot encoded border')

### Cấu hình Mạng Phân đoạn
Quá trình chuẩn bị dữ liệu cho phân đoạn đã hoàn tất. Tiếp theo, chúng ta cấu hình một mạng phân đoạn bằng cách chỉ định các tham số của SegConfig. Ví dụ, bạn có thể tăng giá trị train_epochs để đạt được kết quả tốt hơn, mặc dù điều này sẽ kéo dài thời gian tính toán. (Điều này thường đúng với giá trị fraction lớn.)

In [ ]:
relative_weights = [1.0,1.0,5.0] # Relative weight of background, foreground and border class for training

config = SegConfig(X, unet_kern_size=3, relative_weights = relative_weights,
                   train_steps_per_epoch=400, train_epochs=3, train_loss='seg', batch_norm=True, 
                   train_batch_size=128, unet_n_first = 32, unet_n_depth=4)


# Let's look at the parameters stored in the config-object.
# a name used to identify the model
model_name = 'seg_baseline'
# the base directory in which our model will live
basedir = 'models'
# We are now creating our network model.
seg_model = Seg(config, model_name, basedir=basedir)
vars(config)

In [ ]:
seg_model.train(X, Y, (X_val, Y_val))

Tính ngưỡng tốt nhất trên các ảnh xác thực (để tối ưu hóa điểm Precision Trung bình). Ngưỡng này sẽ được sử dụng để tạo mặt nạ cứng (hard masks) từ các ảnh xác suất dự đoán trên các ảnh kiểm tra.

In [ ]:
threshold=seg_model.optimize_thresholds(X_val_normalized.astype(np.float32), val_masks)

### Dự đoán trên các ảnh kiểm tra để thu được kết quả phân đoạn

In [ ]:
predicted_images, precision_result=seg_model.predict_label_masks(X_test_normalized, test_masks, threshold)
print("Average precision over all test images at IOU = 0.5: ", precision_result)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(predicted_images[22])
plt.title('Prediction')
plt.subplot(1,2,2)
plt.imshow(test_masks[22])
plt.title('Ground Truth')